<center>
<a href="http://www.ai.univ-paris8.fr/index.php?page=membres&member=ap" ><img src="http://www.ai.univ-paris8.fr/~boubchir/logo_liasd.jpeg" style="float:left; max-width: 250px; display: inline" alt="INSA"/></a> 

<a href="http://www.univ-paris8.fr/Master-MIASHS-Big-Data-et-fouille-de-donnees" ><img src="https://autourdemafac.files.wordpress.com/2016/01/p81.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

<center>
<a href="https://www.tripadvisor.co.uk/ " ><img src="https://static.tacdn.com/img2/langs/en_UK/branding/rebrand/TA_logo_primary.svg" style="float:right; max-width: 300px; display: inline" alt="IMT"/> </a>
</center>
# Grand corpus, données des réseaux sociaux
# Projet final : apprentissage pour opinion mining (avis)

L'objectif de ce projet est d'analyser une grande quantité de données (avis Hôtel et restaurant) en deux langues Français et Anglais via Tripadvisor.
Afin d’avoir model de classification des avis en deux classes ( Hôtel ou Restaurant).

In [1]:
# Import all libraries needed for the project
from pandas import DataFrame, read_csv , json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
import time
 # Enable inline plotting
%matplotlib inline

In [2]:
# Python version, Pandas version, Matplotlib version
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.2 |Anaconda custom (32-bit)| (default, Sep 20 2017, 13:35:58) [MSC v.1900 32 bit (Intel)]
Pandas version 0.20.3
Matplotlib version 2.1.0


# 1- Collection des Corpus 

Utilisant Scrapy 1.4 , pour scrapy les avis.

In [3]:
# #######################################################################################
# les coprus de chaque étudinat afin d'avoir Grand Corpus de déférent langues et villes
# Load Data 
# Corpus Abdelkader 
debut = time.time()

dfa_hotel_eng = pd.read_json('corpus/en_hotel_london.json')
dfa_resto_eng = pd.read_json('corpus/en_res_london.json')
dfa_resto_fr = pd.read_json('corpus/fr_rest_londres.json')
# 1 class is resto  0 is not 
dfa_hotel_eng['classe'] = 0  
dfa_resto_eng['classe'] = 1 
dfa_resto_fr['classe'] = 1 
    
# Corpus  Roxane
dfr_resto_eng = pd.read_json('corpus/resto_eng.json')
dfr_hotel_eng = pd.read_json('corpus/hotel_eng.json')
dfr_resto_fr = pd.read_json('corpus/resto_fr.json')
dfr_hotel_fr = pd.read_json('corpus/hotel_fr.json')
# 1 class is resto  0 is not 
dfr_resto_eng['classe'] = 1
dfr_resto_fr['classe'] = 1
dfr_hotel_eng['classe'] = 0
dfr_hotel_fr['classe'] = 0

frames = [dfr_resto_eng,dfr_resto_fr,dfr_hotel_eng, dfr_hotel_fr,dfa_hotel_eng,dfa_resto_eng,dfa_resto_fr]
data= pd.concat(frames)
fin = time.time()
print(fin-debut)

15.51488733291626


In [4]:
# Check data type of the columns
data.dtypes

classe      int64
content    object
date       object
rating     object
title      object
dtype: object

In [5]:
# les 5 premiers lignes du Data
data.head()

,classe,content,date,rating,title
0,1,The place is really cool for breakfast or brun...,"December 17, 2017",ui_bubble_rating bubble_50,Tasty food
1,1,We visited and were definitely not disappointe...,"November 3, 2017",ui_bubble_rating bubble_50,Amazing lunch!
2,1,This is the main restaurant at floor 23 of the...,"November 1, 2017",ui_bubble_rating bubble_40,UNIQUE EXPERIENCE
3,1,Service: Outstanding. The staff was incredibly...,"November 2, 2017",ui_bubble_rating bubble_40,A Delight **
4,1,We recently visited Ciel Bleu and had the Expe...,"November 17, 2017",ui_bubble_rating bubble_50,Extraordinary experience


Pour le projet on va utiliser juste les deux colonnes " content" et "class"

# 2- Préparation des corpus

In [6]:
# We can delete columns
# supprimer les colonnes non important
del data['rating']
del data['title']
del data['date']

In [7]:
# les 5 premiers lignes du Data, après la suppression
data.head()

,classe,content
0,1,The place is really cool for breakfast or brun...
1,1,We visited and were definitely not disappointe...
2,1,This is the main restaurant at floor 23 of the...
3,1,Service: Outstanding. The staff was incredibly...
4,1,We recently visited Ciel Bleu and had the Expe...


In [8]:
# drop duplicate reviews
# supprimer les avis en doublant, s'il existe par erreurs,  
data=data.drop_duplicates(subset='content', keep=False, inplace=False)

In [9]:
# all reviews 
print('Nombre des avis est :',len(data['content']))

Nombre des avis est : 229281


In [10]:
# unique reviews 
print('Nombre des avis unique est :',len(data['content'].unique()))

Nombre des avis unique est : 229281


In [11]:
# remove characters and stoplist words, then generate dictionary of unique words
# supprimer les sympoles non important
data['content'].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',inplace=True,regex=True)

In [12]:
# converte to minuscule
data['content']=data['content'].str.lower()
data.head()

,classe,content
0,1,the place is really cool for breakfast or brun...
1,1,we visited and were definitely not disappointe...
2,1,this is the main restaurant at floor of the...
3,1,service outstanding the staff was incredibly...
4,1,we recently visited ciel bleu and had the expe...


In [13]:
# suppression des mots non important,  stop word fr et an 
# https://www.ranks.nl/stopwords/  site web des stop word 
#print(stop_en)   pour afficher la liste des mots

# Stop Word english
stop_en = stopwords.words('english')
data['content']= data['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_en)]))
# Stop Word french
stop_fr = stopwords.words('french') 
data['content'] = data['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_fr)]))
# afficher les 5 premiers lignes
data.head()

,classe,content
0,1,place really cool breakfast brunch price moder...
1,1,visited definitely disappointed waited table b...
2,1,main restaurant floor okura hotel felt guilty ...
3,1,service outstanding staff incredibly polite pr...
4,1,recently visited ciel bleu experience tasting ...


In [14]:
# préparation des 3 data, train, validate, Test 
dfTrain, dfValidate, dfTest = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])
print('Corpus continet : ',len(data))
print ('Pour apprentissage 60% :',len(dfTrain))
print ('Pour Validation 20% :',len(dfValidate))
print ('Pour Test 20% :',len(dfTest))

Corpus continet :  229281
Pour apprentissage 60% : 137568
Pour Validation 20% : 45856
Pour Test 20% : 45857


In [15]:
# Learn vocabulary and idf, return term-document matrix.
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(dfTrain['content'])
idf = vectorizer.idf_
print (dict(zip(vectorizer.get_feature_names(), idf)))

{'aa': 10.192823558786685, 'aaa': 10.752439346722108, 'aaaah': 11.733268599733835, 'aaaahh': 12.138733707841999, 'aaaahs': 12.138733707841999, 'aaaand': 12.138733707841999, 'aaaargh': 12.138733707841999, 'aaah': 12.138733707841999, 'aaargh': 12.138733707841999, 'aaarrgghh': 12.138733707841999, 'aaawww': 12.138733707841999, 'aachen': 12.138733707841999, 'aadam': 12.138733707841999, 'aadlers': 12.138733707841999, 'aadooooooore': 12.138733707841999, 'aadu': 9.8874419092355037, 'aadvantage': 12.138733707841999, 'aadvisor': 12.138733707841999, 'aafe': 12.138733707841999, 'aagin': 12.138733707841999, 'aah': 12.138733707841999, 'aahs': 11.222442975967844, 'aain': 12.138733707841999, 'aaldering': 12.138733707841999, 'aalders': 12.138733707841999, 'aaliable': 12.138733707841999, 'aalso': 12.138733707841999, 'aamazing': 12.138733707841999, 'aamiaiskahvila': 12.138733707841999, 'aamzing': 12.138733707841999, 'aan': 11.445586527282053, 'aapams': 12.138733707841999, 'aardappelen': 12.13873370784199

In [16]:
# afficher les 100 top mots
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dfTrain['content'])
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 100
top_features = [features[i] for i in indices[:top_n]]
print (top_features)

['하나만', 'inovation', 'inouã', 'inordinate', 'inoperative', 'inntels', 'inntelhotel', 'innss', 'innovativeness', 'innovated', 'innonde', 'innocent', 'innner', 'innn', 'innkeepers', 'innkeeper', 'innings', 'inovated', 'inox', 'inseparable', 'inpeccable', 'inself', 'insecurities', 'insects', 'inscrivez', 'inscrivant', 'inscrire', 'insatiable', 'insalada', 'inroom', 'inredibly', 'inquisitive', 'inquiets', 'inquiet', 'inputs', 'inpolite', 'innhotel', 'innermost', 'innards', 'innapropriatly', 'initier', 'initiatique', 'initiation', 'initiated', 'initialy', 'init', 'inis', 'inintã', 'iniitial', 'inigual', 'inhibitions', 'inhibited', 'inheritence', 'inheritance', 'inherit', 'initmate', 'initself', 'inject', 'inluding', 'innapropriate', 'innaccã', 'inmyexpectations', 'inmersion', 'inmediatelly', 'inly', 'inloved', 'injected', 'inlet', 'inlcudes', 'inlcuded', 'inlcluded', 'inlaws', 'injure', 'insensitive', 'inset', 'inhave', 'intant', 'insuring', 'insures', 'insured', 'insulates', 'insular', 'in

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents 
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(dfTrain['content'])
# summarize
print(vectorizer.vocabulary_)

{'lunch': 31577, 'dinner': 15486, 'booked': 6363, 'surprise': 52086, 'treat': 55016, 'fine': 20550, 'opted': 36967, 'carte': 8497, 'menu': 33182, 'heard': 24980, 'much': 34646, 'whilst': 58805, 'hubby': 26059, 'special': 50037, 'set': 47789, 'excellent': 18975, 'value': 56889, 'serving': 47759, 'staff': 50588, 'attentive': 3741, 'knowledgeable': 29473, 'happy': 24651, 'chat': 9328, 'restaurant': 44807, 'food': 21236, 'sommelier': 49598, 'equally': 18318, 'helpful': 25138, 'interesting': 27703, 'wine': 59072, 'choices': 9834, 'always': 1716, 'think': 53697, 'lunchtime': 31594, 'hectic': 25038, 'turn': 55488, 'detrimental': 15078, 'atmosphere': 3595, 'would': 59453, 'say': 46853, 'leisurely': 30362, 'rushed': 46167, 'way': 58393, 'lovely': 31421, 'buzz': 7726, 'budget': 7363, 'hotel': 25899, 'location': 31043, 'room': 45826, 'standard': 50687, 'short': 48227, 'walk': 58106, 'st': 50557, 'pancras': 37965, 'station': 50805, 'main': 31920, 'tube': 55393, 'accessing': 281, 'london': 31181, '

# 3- Apprentissage et Test

In [18]:
from sklearn import naive_bayes
y = data.classe
X = vectorizer.fit_transform(data.content)

In [19]:
#y = data.classe

In [20]:
#X = vectorizer.fit_transform(data.content)

In [21]:
print (y.shape)
print(X.shape)

(229281,)
(229281, 76525)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=41)

In [23]:
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
# 0.99456823572815967   ---- 41
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, clf.predict_proba(X_test,)[:,1])

0.99456823572815967

In [25]:
# hotel = 0
# resto = 1
#avis=np.array(["Mais l'autre véritable es de marche d'Oxford Stree"])
#avis_vec=vectorizer.transform(avis)
#print (clf.predict(avis_vec))

In [75]:
# pour avoir des avis      https://www.tripadvisor.co.uk/Hotels-g186338-London_England-Hotels.html
#                          https://www.tripadvisor.co.uk/Restaurants-g186338-London_England.html
# saisir votre avis si : 
# hotel = 0         
# resto = 1          
text = input()
avis=np.array([text])
avis_vec=vectorizer.transform(avis)
print('votre avis est :', clf.predict(avis_vec))  

Un Très bon interface, de bonne qualités sur la page. Proximité du métro 400m. Dans une partie proche du centre, avec un quartier chic et calme. Les chambres sont spacieuses, avec la possibilité de venir en famille .
votre avis est : [0]


In [ ]:
# ces avis d'un hotel : Naive classe comme resto mais Rondom Forest classe correcte
#Cadre agréable avec un personnel très agréable à recommander avec des chambres confortables et pratiques choses pas évidentes pour des hôtels de qualité supérieure. Le service restaurant digne des grandes enseignes de la restauration j'ai été très surpri
#Très charmant,personnel accueillant chambres très spatieuse, la restauration est excellente. Beaucoup de visites à faire aux alentours , la ville est très sympa. Idéal pour voyage en couple Seul bémol la météo, pensez à réserver pour la bonne période.

In [27]:
fin=time.time()
print(fin-debut)

507.25870633125305


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.3)

In [29]:
from sklearn.tree import DecisionTreeClassifier

In [30]:
dtree = DecisionTreeClassifier()

In [31]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
predictions = dtree.predict(X_test)

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

In [34]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[22954  1418]
 [ 1292 43121]]


             precision    recall  f1-score   support

          0       0.95      0.94      0.94     24372
          1       0.97      0.97      0.97     44413

avg / total       0.96      0.96      0.96     68785



In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
rfc = RandomForestClassifier(n_estimators=200)

In [37]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [41]:
rfc_pred = rfc.predict(X_test)

In [42]:
print(confusion_matrix(y_test,rfc_pred))
print('\n')
print(classification_report(y_test,rfc_pred))

[[23464   908]
 [  357 44056]]


             precision    recall  f1-score   support

          0       0.99      0.96      0.97     24372
          1       0.98      0.99      0.99     44413

avg / total       0.98      0.98      0.98     68785



In [44]:
data['classe'].value_counts()

1    147848
0     81433
Name: classe, dtype: int64

In [74]:
# saisir votre avis si : 
# hotel = 0         
# resto = 1
text = input()
avis=np.array([text])
avis_vec=vectorizer.transform(avis)
print('votre avis est :', rfc.predict(avis_vec)) 

Un Très bon interface, de bonne qualités sur la page. Proximité du métro 400m. Dans une partie proche du centre, avec un quartier chic et calme. Les chambres sont spacieuses, avec la possibilité de venir en famille .
votre avis est : [0]
